In [ ]:
!pip install requests beautifulsoup4 selenium transformers flask-ngrok --quiet
!apt-get update
!apt install chromium-chromedriver --quiet
!pip install flask flask_tunnel transformers
!pip install lxml[html_clean] --quiet






Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists...
Building dependency tree...
Reading state inf

In [ ]:
!pip install requests_html

In [13]:
!pip install nest_asyncio requests-html beautifulsoup4 sentence-transformers


In [47]:
!pip install chromedriver_autoinstaller

In [56]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the SentenceTransformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Function to fetch all hyperlinks from the main website
def fetch_hyperlinks(base_url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        links = [urljoin(base_url, a['href']) for a in soup.find_all('a', href=True)]
        return list(set(links))  # Remove duplicates
    except Exception as e:
        print(f"Error fetching hyperlinks: {e}")
        return []

# Function to scrape content from a URL
def scrape_content(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        content = []

        # Extract headers and paragraphs
        for header in soup.find_all(['h1', 'h2', 'h3']):
            section_content = f"Header: {header.text.strip()}"
            paragraphs = [p.text.strip() for p in header.find_next_siblings('p')]
            section_content += "\n".join(paragraphs)
            content.append(section_content)

        return content
    except Exception as e:
        print(f"Error scraping content from {url}: {e}")
        return []

# Function to find the most relevant content
def find_relevant_content(question, content):
    question_embedding = model.encode([question])
    content_embeddings = model.encode(content)
    similarities = cosine_similarity(question_embedding, content_embeddings)

    # Sort content by relevance
    sorted_indices = np.argsort(similarities[0])[::-1]
    top_indices = sorted_indices[:3]  # Get top 3 relevant sections
    relevant_content = [content[i] for i in top_indices]
    return relevant_content

# Main function to scrape a website and answer a question
def main():
    # Define websites
    websites = {
        '1': 'https://segment.com/docs/?ref=nav',
        '2': 'https://docs.mparticle.com/',
        '3': 'https://docs.lytics.com/',
        '4': 'https://docs.zeotap.com/home/en-us/'
    }

    print("Please select a website from the following options:")
    print("1. Segment Documentation")
    print("2. mParticle Documentation")
    print("3. Lytics Documentation")
    print("4. Zeotap Documentation")

    choice = input("Enter the number corresponding to the website you want to query: ")
    base_url = websites.get(choice)

    if not base_url:
        print("Invalid choice. Please try again.")
        return

    print(f"\nFetching content from {base_url}...")

    # Fetch hyperlinks from the base URL
    links = fetch_hyperlinks(base_url)

    if not links:
        print("No hyperlinks found. Exiting.")
        return

    print(f"\nFound {len(links)} hyperlinks. Scraping content...")

    # Scrape content from the base URL and all sub-links
    all_content = []
    for link in links[:20]:  # Limit to the first 20 links for efficiency
        all_content.extend(scrape_content(link))

    print(f"\nScraped {len(all_content)} sections of content.")

    # Ask the user a question
    question = input("\nEnter your question: ")

    # Find the most relevant content
    relevant_content = find_relevant_content(question, all_content)

    # Format and display the response
    print("\n### Answer:")
    for i, section in enumerate(relevant_content, 1):
        print(f"{i}. {section.split('Header: ')[-1].strip()}")
        print()

    # Suggest the main link for further exploration
    print(f"To explore more, visit: {base_url}")

# Run the program
if __name__ == "__main__":
    main()


Please select a website from the following options:
1. Segment Documentation
2. mParticle Documentation
3. Lytics Documentation
4. Zeotap Documentation
Enter the number corresponding to the website you want to query: 3

Fetching content from https://docs.lytics.com/...

Found 37 hyperlinks. Scraping content...

Scraped 379 sections of content.

Enter your question: How do I build an audience segment in Lytics

### Answer:
1. Compositional InsightsCompositional Insights help you understand the makeup of your audiences. This information can answer, “what makes an audience unique?” and “How should I target these users differently?”
Compositional Insights make it easier for you to create effective segmentation strategies. They compare data fields within audiences to surface which attributes drive user behavior. Compositional Insights currently include the following:
This Insight compares high-value audiences on their levels of engagement. This can take the form of comparing Lytic's out-of-